In [132]:
import pandas as pd
import glob
import matplotlib.pyplot as plt
import numpy as np 

import pvlib
from pvlib import clearsky, atmosphere, solarposition
from pvlib.location import Location
from pvlib.iotools import read_tmy3

import warnings
warnings.filterwarnings("ignore")


# 0 - Importing data

In [133]:
GHI_df = pd.DataFrame()
years = np.linspace(2016,2021,6, dtype= int)
file_path = './GHI_dataset/raw_data/GHI_raw_'

for year in years:
    file_path_temp = file_path + str(year) + '.csv'
    df_temp = pd.read_csv(file_path_temp)
    GHI_df = pd.concat([GHI_df, df_temp], axis = 0)

GHI_df['datetime'] = pd.to_datetime(GHI_df['datetime'], format="%Y-%m-%d %H:%M:%S")

GHI_df.set_index('datetime', inplace = True)

In [134]:
print("Number of GHI_df measurements: " + str(GHI_df.shape[0]))

print("Number of NA: " + str(GHI_df['GHI'].isnull().sum()))

GHI_df.head(10)

Number of GHI_df measurements: 17297280
Number of NA: 717723


,GHI
datetime,
2016-01-08 00:00:00,2.923
2016-01-08 00:00:10,2.957
2016-01-08 00:00:20,2.945
2016-01-08 00:00:30,2.938
2016-01-08 00:00:40,2.936
2016-01-08 00:00:50,2.941
2016-01-08 00:01:00,2.934
2016-01-08 00:01:10,2.941
2016-01-08 00:01:20,2.961


## Step 1 - Removal of missing values

In [135]:
# Define a threshold for the maximum number of consecutive missing values allowed
threshold = 1

# Identify and remove high-density consecutive missing values
def remove_consecutive_nan(df, column, threshold):
    bool_series = df[column].isnull()
    df['block'] = (bool_series.diff(1) != 0).astype('int').cumsum()
    df = df[~((df[column].isnull()) & (df.groupby('block')['block'].transform('size') > threshold))]
    df = df.drop('block', axis=1)
    return df

GHI_df = remove_consecutive_nan(GHI_df, 'GHI', threshold)

print('Number of GHI measurements to interpolate: ' + str(GHI_df['GHI'].isnull().sum()))

Number of GHI measurements to interpolate: 11683


In [136]:
GHI_df['GHI'] = GHI_df['GHI'].interpolate()

print('Number of NaNs: ' + str(GHI_df['GHI'].isnull().sum()))
print("Number of GHI measurements: " + str(GHI_df.shape[0]))

Number of NaNs: 0
Number of GHI measurements: 16591240


## Step 2 - Identification and removal of outliers

In [137]:
## Removal of outliers
GHI_max = 1000
GHI_min = 0

GHI_df = GHI_df[(GHI_df["GHI"] < GHI_max) & (GHI_df["GHI"] > GHI_min)]
print("Number of GHI measurements: " + str(GHI_df.shape[0]))

Number of GHI measurements: 16241644


## Step 3 - Clear sky global horizontal irradiance (GHIcs)

In [138]:
#Clear sky GHI calculation

latitude = 46.518
longitude = 6.565
time_zone = 'Europe/Zurich'
altitude = 400
place = 'Ecublens'
frequency = '10S'

tus = Location(latitude, longitude, time_zone, altitude, place)

cs = tus.get_clearsky(GHI_df.index)
GHI_df['GHIcs'] = cs.ghi


## Step 4 - Removal of night measurements

In [139]:
# Night is assumed when GHIcs is inferior to 30 W/m2

nb_night_measurements = int(100*GHI_df[GHI_df['GHIcs']<30].shape[0] / GHI_df.shape[0])

GHI_df = GHI_df[GHI_df["GHIcs"] > 30]

print("Percentage of GHI night measurements: " + str(nb_night_measurements) + "%")
print("Number of GHI measurements: " + str(GHI_df.shape[0]))

Percentage of GHI night measurements: 53%
Number of GHI measurements: 7490668


## Step 5 - Clear sky index (kcs)

In [140]:
#Calculating the clear sky index - k

GHI_df['k'] = GHI_df["GHI"] / GHI_df["GHIcs"]

#Night measurement have GHIcs = 0 => k=inf ; when it happens, we set k to 0
GHI_df.replace([np.inf, -np.inf], 0, inplace= True)

GHI_df.head(5)

,GHI,GHIcs,k
datetime,,,
2016-01-08 07:59:00,15.394,30.209756,0.509570
2016-01-08 07:59:10,15.447,30.471824,0.506927
2016-01-08 07:59:20,15.472,30.734662,0.503406
2016-01-08 07:59:30,15.542,30.998266,0.501383
2016-01-08 07:59:40,15.539,31.262629,0.497047


## Step 6 - Calculation of finite-difference

In [141]:
def FirstOrderBackdDiff (serie, h):
    y_t0 = serie.values.flatten()[1:]
    y_t1 = serie.values.flatten()[0:-1]
    return np.concatenate([[np.nan], (y_t0-y_t1)/(h)])

def SecondOrderBackDiff (serie, h):
    y_t0 = serie.values.flatten()[2:]
    y_t1 = serie.values.flatten()[1:-1]
    y_t2 = serie.values.flatten()[:-2]
    return np.concatenate([[np.nan], [np.nan], (y_t0-2*y_t1+y_t2)/((h)**2)])
    
def ThirdOrderBackDiff(serie, h):
    y_t0 = serie.values.flatten()[3:]
    y_t1 = serie.values.flatten()[1:-2]
    y_t2 = serie.values.flatten()[2:-1]
    y_t3 = serie.values.flatten()[3:]
    return np.concatenate([[np.nan], [np.nan], [np.nan], (y_t0-3*y_t1+3*y_t2-y_t3)/(h**3)])

h = 10 #10s

GHI_df["GHI_d1"] = FirstOrderBackdDiff(GHI_df['GHI'], h)
GHI_df["GHIcs_d1"] = FirstOrderBackdDiff(GHI_df['GHIcs'], h)
GHI_df["k_d1"] = FirstOrderBackdDiff(GHI_df['k'], h)

GHI_df["GHI_d2"] = SecondOrderBackDiff(GHI_df['GHI'], h)
GHI_df["GHIcs_d2"] = SecondOrderBackDiff(GHI_df['GHIcs'], h)
GHI_df["k_d2"] = SecondOrderBackDiff(GHI_df['k'], h)

GHI_df["GHI_d3"] = ThirdOrderBackDiff(GHI_df['GHI'], h)
GHI_df["GHIcs_d3"] = ThirdOrderBackDiff(GHI_df['GHIcs'], h)
GHI_df["k_d3"] = ThirdOrderBackDiff(GHI_df['k'], h)

GHI_df = GHI_df.dropna()

In [127]:
GHI_df.head(5)

,GHI,GHIcs,k,GHI_d1,GHIcs_d1,k_d1,GHI_d2,GHIcs_d2,k_d2,GHI_d3,GHIcs_d3,k_d3
datetime,,,,,,,,,,,,
2016-01-08 07:59:30,15.542,30.998266,0.501383,0.0070,0.026360,-0.000202,0.00045,0.000008,0.000015,0.000075,0.000789,-0.000011
2016-01-08 07:59:40,15.539,31.262629,0.497047,-0.0003,0.026436,-0.000434,-0.00073,0.000008,-0.000023,0.000210,0.000791,-0.000006
2016-01-08 07:59:50,15.569,31.527745,0.493819,0.0030,0.026512,-0.000323,0.00033,0.000008,0.000011,-0.000009,0.000793,-0.000013
2016-01-08 08:00:00,15.644,31.793610,0.492049,0.0075,0.026587,-0.000177,0.00045,0.000007,0.000015,0.000090,0.000795,-0.000010
2016-01-08 08:00:10,15.677,32.060216,0.488986,0.0033,0.026661,-0.000306,-0.00042,0.000007,-0.000013,0.000225,0.000798,-0.000005


## Step 7 - Adding seasonality

In [142]:
def norm_sin_hour(x):
    return np.sin(x*np.pi/24)

def norm_sin_day(x):
    return np.sin(x*np.pi/365)

def norm_sin_month(x):
    return np.sin(x*np.pi/12)


GHI_df["month"] = norm_sin_month(GHI_df.index.month).copy()

GHI_df["day"] = norm_sin_day(GHI_df.index.dayofyear).copy()

GHI_df["hour"] = norm_sin_hour(GHI_df.index.hour).copy()

GHI_df.head(5)

,GHI,GHIcs,k,GHI_d1,GHIcs_d1,k_d1,GHI_d2,GHIcs_d2,k_d2,GHI_d3,GHIcs_d3,k_d3,month,day,hour
datetime,,,,,,,,,,,,,,,
2016-01-08 07:59:30,15.542,30.998266,0.501383,0.0070,0.026360,-0.000202,0.00045,0.000008,0.000015,0.000075,0.000789,-0.000011,0.258819,0.068802,0.793353
2016-01-08 07:59:40,15.539,31.262629,0.497047,-0.0003,0.026436,-0.000434,-0.00073,0.000008,-0.000023,0.000210,0.000791,-0.000006,0.258819,0.068802,0.793353
2016-01-08 07:59:50,15.569,31.527745,0.493819,0.0030,0.026512,-0.000323,0.00033,0.000008,0.000011,-0.000009,0.000793,-0.000013,0.258819,0.068802,0.793353
2016-01-08 08:00:00,15.644,31.793610,0.492049,0.0075,0.026587,-0.000177,0.00045,0.000007,0.000015,0.000090,0.000795,-0.000010,0.258819,0.068802,0.866025
2016-01-08 08:00:10,15.677,32.060216,0.488986,0.0033,0.026661,-0.000306,-0.00042,0.000007,-0.000013,0.000225,0.000798,-0.000005,0.258819,0.068802,0.866025


## Step 8 - Sampling

In [160]:
import os
sampling_frequencies = ["1T", "5T", "10T", "15T", #Removed 10s to be able to push to Git
                        "30T", "45T", "1H", "2H", "4H", 
                        "6H", "12H", "24H", "48H", "72H",
                        "4D", "5D", "6D", "7D"]
sampling_frequencies_eng = ["1mn", "5mn", "10mn", "15mn", #Removed 10s to be able to push to Git
                            "30mn", "45mn", "1h", "2h", "4h", 
                            "6h", "12h", "24h", "48h", "72h", 
                            "4days", "5days", "6days", "7days"]

root = "./GHI_dataset/cleaned_sampled/"

for i, frequency in enumerate(sampling_frequencies):

    print(frequency)
    
    df_sampled_temp = GHI_df.resample(frequency).mean().copy()
    df_sampled_temp = df_sampled_temp.dropna()
    df_sampled_temp.reset_index(inplace = True)

    df_sampled_temp['year'] = df_sampled_temp['datetime'].dt.year
    years = df_sampled_temp['year'].unique()

    path_freq = os.path.join(root, frequency)
    os.mkdir(path_freq)

    # Loop over each year and export the data
    for year in years:
        
        print("--" + str(year))
        path_year = os.path.join(path_freq, str(year))
        os.mkdir(path_year)
    
        file_path= path_year + '/GHI_sampled_' + str(frequency) + '_' + str(year) + '.csv'

        df_year = df_sampled_temp[df_sampled_temp['year'] == year]
        df_year.to_csv(file_path, index=False)

10s
--2016
--2017
--2018
--2019
--2020
--2021
1T
--2016
--2017
--2018
--2019
--2020
--2021
5T
--2016
--2017
--2018
--2019
--2020
--2021
10T
--2016
--2017
--2018
--2019
--2020
--2021
15T
--2016
--2017
--2018
--2019
--2020
--2021
30T
--2016
--2017
--2018
--2019
--2020
--2021
45T
--2016
--2017
--2018
--2019
--2020
--2021
1H
--2016
--2017
--2018
--2019
--2020
--2021
2H
--2016
--2017
--2018
--2019
--2020
--2021
4H
--2016
--2017
--2018
--2019
--2020
--2021
6H
--2016
--2017
--2018
--2019
--2020
--2021
12H
--2016
--2017
--2018
--2019
--2020
--2021
24H
--2016
--2017
--2018
--2019
--2020
--2021
48H
--2016
--2017
--2018
--2019
--2020
--2021
72H
--2016
--2017
--2018
--2019
--2020
--2021
4D
--2016
--2017
--2018
--2019
--2020
--2021
5D
--2016
--2017
--2018
--2019
--2020
--2021
6D
--2016
--2017
--2018
--2019
--2020
--2021
7D
--2016
--2017
--2018
--2019
--2020
--2021
